Python 3.11.2에 연결됨

In [1]:
import requests
import json
import math
import time

HOST = "http://127.0.0.1:56123"

REGISTRATION_URL = f"{HOST}/api/v4/rest-auth/registration/"
LOGIN_URL = f"{HOST}/api/v4/rest-auth/login/"
LOGOUT_URL = f"{HOST}/api/v4/rest-auth/logout/"
UPLOAD_URL = f"{HOST}/api/v4/upload"
DIVIDE_URL = f"{HOST}/api/v4/parser/divide"
PARSE_URL = f"{HOST}/api/v4/parser/parse"
ANNOTATE_URL = f"{HOST}/api/v4/annotator/annotate"
CORPUSES_URL = f"{HOST}/api/v4/corpuses/"

url = "http://127.0.0.1:56123"


In [2]:
join_membership = {
    "username": "abc111",
    "email": "abcdefghi12@naver.com",
    "password1": "testaaaaaa!",
    "password2": "testaaaaaa!"
}

join_membership_response = requests.post(REGISTRATION_URL, data=join_membership)

login = {
    "username": "abc111",
    "email": "abcdefghi12@naver.com",
    "password": "testaaaaaa!"
}

login_response= requests.post(LOGIN_URL, json=login)
login_data = login_response.json()
token = login_data.get('key')

login_key = {
    'Authorization': f'Token {token}'
}


In [4]:
texts = [
    {
        "original_text": "Die Alpen erstrecken sich über mehrere Länder Europas und bieten spektakuläre Landschaften. Skifahren, Wandern und Klettern sind beliebte Aktivitäten in dieser Region"
    },
    {
        "original_text": "Der Rhein ist einer der längsten Flüsse Europas. Entlang seiner Ufer befinden sich zahlreiche Burgen und Malerische Dörfer, die Touristen aus der ganzen Welt anziehen."
    },
    {
        "original_text": "München, bekannt für das Oktoberfest, ist eine Stadt mit reicher Kultur. Die Altstadt, der Englische Garten und das Schloss Nymphenburg sind beliebte"
    },
    {
        "original_text": "Der Schwarzwald ist ein großes Waldgebiet im Südwesten Deutschlands. Die Region ist berühmt für ihre dichten Wälder, Malerischen Dörfer und die berühmte Kuck"

    },
    {
        "original_text": "Die Insel Rügen ist die größte Insel Deutschlands und liegt in der Ostsee. Sie ist bekannt für ihre weißen Kreidefelsen, die im Nationalpark Jasmund besonders beeindruckend sind"

    },
    {
        "original_text": "Die Wartburg, eine mittelalterliche Burg in Thüringen, ist ein UNESCO-Weltkulturerbe. Hier übersetzte Martin Luther das Neue Testament der Bibel ins Deutsche"
    }
]

In [5]:
time_list = []
annotate_list = []
annotators = ["chatgpt_ft0", "chatgpt_gpt-3.5-turbo-untrained_0","chatgpt_gpt-3.5-turbo-pretrained_0"]

for annotator_name in annotators:
    for text in texts:
        upload_response = requests.post(UPLOAD_URL, json=text, headers=login_key)
        corpus_id = upload_response.json().get('corpus_id')

        divide_data = {
            "corpus_id": corpus_id,
            "divide_options": json.dumps({"p_delims": ["\n"]})
        }
        divide_response = requests.post(DIVIDE_URL, json=divide_data, headers=login_key)

        parse_data = {
            "corpus_id": corpus_id,
            "parse_options": json.dumps({"t_delims": " \t\n\r\u000b\f"})
        }
        parse_response = requests.post(PARSE_URL, json=parse_data, headers=login_key)

        start = time.time()

        annotate_data = {
            "corpus_id": corpus_id,
            "annotate_options": json.dumps({
                "lang_from": "German",
                "lang_to": "English",
                "annotator_name": annotator_name
            })
        }
        annotate_response = requests.post(ANNOTATE_URL, json=annotate_data, headers=login_key)
        
        end = time.time()
        print(annotator_name)
        print(f"{end - start:.5f} sec")
        time_list.append(end - start)
        annotate_list.append(annotator_name)


chatgpt_ft0
4.26056 sec
chatgpt_ft0
3.10497 sec
chatgpt_ft0
2.62273 sec
chatgpt_ft0
2.53498 sec
chatgpt_ft0
2.76329 sec
chatgpt_ft0
2.96308 sec
chatgpt_gpt-3.5-turbo-untrained_0
2.49897 sec
chatgpt_gpt-3.5-turbo-untrained_0
3.81171 sec
chatgpt_gpt-3.5-turbo-untrained_0
2.35203 sec
chatgpt_gpt-3.5-turbo-untrained_0
2.76561 sec
chatgpt_gpt-3.5-turbo-untrained_0
2.54061 sec
chatgpt_gpt-3.5-turbo-untrained_0
2.97267 sec
chatgpt_gpt-3.5-turbo-pretrained_0
5.21870 sec
chatgpt_gpt-3.5-turbo-pretrained_0
3.05281 sec
chatgpt_gpt-3.5-turbo-pretrained_0
2.45449 sec
chatgpt_gpt-3.5-turbo-pretrained_0
2.71919 sec
chatgpt_gpt-3.5-turbo-pretrained_0
2.56766 sec
chatgpt_gpt-3.5-turbo-pretrained_0
2.12853 sec


In [10]:
corpuses_respons = requests.get(CORPUSES_URL, headers=login_key)
corpuse_history = corpuses_respons.json()

texts_len = len(texts) # 6
num = int(corpuse_history[-1].get('corpus_id'))  # 마지막 코퍼스의 ID 가져오기
first = num - (texts_len*len(annotators))
number2 = 0

for number in range(first, num):
    Unknown_count = 0
    token_sum = 0
    for corpus_record in corpuse_history[number]['corpuses_history']:
        for paragraph in corpus_record['paragraphs']:
            if paragraph['pstate'] == "ANNOTATED":
                for token in paragraph.get('tokens', []):
                    if not token['is_delimiter']:
                        token_sum += 1
                        if token.get('gloss') == "!UNKNOWN":
                            Unknown_count += 1
                            print(token)

    if token_sum > 0:
        Return_rate = ((token_sum - Unknown_count) / token_sum) * 100
        print(number2, "annotator_name:", annotate_list[number2], "time:", round(float(time_list[number2]), 2), ", return_rate:", round(Return_rate, 2))
    else:
        print(number, "time:", round(float(time_list[number2]), 2), ", No tokens to calculate return rate")
    
    number2 += 1


0 annotator_name: chatgpt_ft0 time: 4.26 , return_rate: 100.0
1 annotator_name: chatgpt_ft0 time: 3.1 , return_rate: 100.0
2 annotator_name: chatgpt_ft0 time: 2.62 , return_rate: 100.0
3 annotator_name: chatgpt_ft0 time: 2.53 , return_rate: 100.0
4 annotator_name: chatgpt_ft0 time: 2.76 , return_rate: 100.0
5 annotator_name: chatgpt_ft0 time: 2.96 , return_rate: 100.0
6 annotator_name: chatgpt_gpt-3.5-turbo-untrained_0 time: 2.5 , return_rate: 100.0
7 annotator_name: chatgpt_gpt-3.5-turbo-untrained_0 time: 3.81 , return_rate: 100.0
8 annotator_name: chatgpt_gpt-3.5-turbo-untrained_0 time: 2.35 , return_rate: 100.0
9 annotator_name: chatgpt_gpt-3.5-turbo-untrained_0 time: 2.77 , return_rate: 100.0
10 annotator_name: chatgpt_gpt-3.5-turbo-untrained_0 time: 2.54 , return_rate: 100.0
11 annotator_name: chatgpt_gpt-3.5-turbo-untrained_0 time: 2.97 , return_rate: 100.0
12 annotator_name: chatgpt_gpt-3.5-turbo-pretrained_0 time: 5.22 , return_rate: 100.0
13 annotator_name: chatgpt_gpt-3.5-tur